In [1]:
import serial
import time
from datetime import datetime
from sqlalchemy import create_engine, exc
from sqlalchemy.orm import sessionmaker
from app import Slot, User, SlotHistory
import pytz

In [2]:
DATABASE_URI = 'sqlite:////Users/jeevanprakash/Documents/TARP/users.db'

In [3]:
# Initialize serial connection
ser = serial.Serial('/dev/cu.usbmodem1101', 9600, timeout=1)
ser.flush()

In [4]:
# Set up database connection
engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
# Function to convert UTC to IST
def utc_to_ist(utc_dt):
    ist_tz = pytz.timezone('Asia/Kolkata')
    return utc_dt.replace(tzinfo=pytz.utc).astimezone(ist_tz)



def update_slot(slot_number, status, user_id=None, booking_method='hardware'):
    slot = session.query(Slot).filter_by(slot_number=slot_number).first()
    if not slot:
        print(f"Creating new slot: {slot_number}")
        slot = Slot(slot_number=slot_number, booked=False)
        session.add(slot)
        session.commit()

    if status == '1' and not slot.booked:
        slot.user_id = user_id if user_id is not None else 3  
        slot.booked = True
        slot.booking_method = booking_method

        # Convert the current time to IST for booking time
        ist_booking_time = utc_to_ist(datetime.utcnow())
        slot.booking_time = ist_booking_time
        session.commit()
        print(f"Slot {slot_number} booked at {ist_booking_time}.")

    elif status == '0' and slot.booked:
        ist_end_time = utc_to_ist(datetime.utcnow())  

        # Record in SlotHistory
        slot_history = SlotHistory(
            slot_number=slot.slot_number,
            user_id=slot.user_id,
            booking_time=slot.booking_time,  # Already in IST
            end_time=ist_end_time
        )
        session.add(slot_history)

        # Delete the slot record
        session.delete(slot)
        session.commit()
        print(f"Slot {slot_number} booking canceled and deleted at {ist_end_time}.")


In [6]:
while True:
    if ser.in_waiting > 0:
        line = ser.readline().decode('utf-8').rstrip()
        print("Received:", line)
        slot_data = line.split(',')
        if len(slot_data) == 2:
            slot_number, status = slot_data
            update_slot(slot_number, status)

Received: Slot 1.1,0
Creating new slot: Slot 1.1
Received: Slot 1.2,0
Creating new slot: Slot 1.2
Received: Slot 1.3,0
Creating new slot: Slot 1.3
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slot 1.2,0
Received: Slot 1.3,0
Received: Slot 1.1,0
Received: Slo


KeyboardInterrupt

